In [1]:
import numpy as np
import os
import torch

from wavenet.model import WaveNet
from wavenet.config import (
    LAYER_SIZE,
    STACK_SIZE,
#     IN_CHANNELS,
    RES_CHANNELS,

    LEARNING_RATE,
    SAMPLE_SIZE,
    SAMPLE_RATE,
    
    DATA_DIR,
    NUM_STEPS,
    MODEL_DIR,
    LOG_DIR,
    TEST_OUTPUT_DIR,
    OUTPUT_DIR,
)

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR
)

from torch.utils.data import DataLoader


In [4]:
X_train = np.load(FEATURE_DIR+'X_train.npy')
y_train = np.load(FEATURE_DIR+'y_train.npy')

X_valid = np.load(FEATURE_DIR+'X_valid.npy')
y_valid = np.load(FEATURE_DIR+'y_train.npy')

In [5]:
IN_CHANNELS = X_train.shape[1]
OUT_CHANNELS = 16

In [12]:
X_train = np.reshape(X_train, (-1, 60, X_train.shape[1]))
x_train_loader = DataLoader(X_train, batch_size=64, shuffle=False)
for features in x_train_loader:
    print(features.shape)
    break


ValueError: cannot reshape array of size 223191444 into shape (60,1)

In [7]:
y_train_loader = DataLoader(y_train, batch_size=64, shuffle=False)
y_train = np.reshape(y_train, (-1, 1, 16))

In [8]:
wavenet = WaveNet(LAYER_SIZE, STACK_SIZE,
                               IN_CHANNELS, RES_CHANNELS, OUT_CHANNELS,
                               lr=LEARNING_RATE)

In [10]:
NUM_STEPS = 1000

In [11]:
total_steps = 0

for features, label in zip(x_train_loader, y_train_loader):
    loss = wavenet.train(features, label)
    total_steps += 1
    print('[{0}/{1}] loss: {2}'.format(total_steps, NUM_STEPS, loss))

    if total_steps > NUM_STEPS:
        break



InputSizeError: Input size has to be larger than receptive_fields
Input size: 1, Receptive fields size: 5115, Output size: -5114

In [11]:
class Trainer:
    def __init__(self):
        self.wavenet = WaveNet(LAYER_SIZE, STACK_SIZE,
                               IN_CHANNELS, RES_CHANNELS,
                               lr=LEARNING_RATE)

    def infinite_batch(self):
        while True:
            for inputs, targets in zip(X_train, y_train):
                yield inputs, targets

    def run(self):
        total_steps = 0

        for inputs, targets in self.infinite_batch():
            loss = self.wavenet.train(inputs, targets)

            total_steps += 1

            print('[{0}/{1}] loss: {2}'.format(total_steps, NUM_STEPS, loss))

            if total_steps > NUM_STEPS:
                break

        self.wavenet.save(MODEL_DIR)

    def model(self):
        return self.wavenet